## This is an experimental notebook to conduct analysis on data scientist positions on Linkedin

In [1]:
import requests
from bs4 import BeautifulSoup
import random
import pandas as pd

In [2]:
## Creating loop to go through all pages
job_list = []
for page_num in range(1,50):
    list_url = f"https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Data%2BScientist&location=San%2BFrancisco%2BBay%2BArea&geoId=90000084&trk=public_jobs_jobs-search-bar_search-submit&start={page_num}" 
    # Getting response request from list 
    response = requests.get(list_url)

    list_data = response.text
    list_soup = BeautifulSoup(list_data, 'html.parser')
    page_jobs = list_soup.find_all("li")
    #10 jobs per page
    #print(len(page_jobs))
    
    ## Get job ID's from each page
    id_list = []

    for job in page_jobs:
        base_card_div = job.find("div", {"class": "base-card"})
        job_id = base_card_div.get("data-entity-urn").split(":")[3]
        id_list.append(job_id)
        
    # For every job with ID, get the information
    for job_id in id_list:
        job_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
        job_response = requests.get(job_url)
        if(job_response.status_code == 200):
            #print(job_response.status_code)
            job_soup = BeautifulSoup(job_response.text, "html.parser")
            job_post = {}
            try:
                job_post["company_name"] = job_soup.find( "a", {"class": "topcard__org-name-link topcard__flavor--black-link"}).text.strip()
            except: 
                job_post["company_name"] = None
            try:
                job_post["job_title"] = job_soup.find( "h2", {"class": "top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title"}).text.strip()
            except: 
                job_post["job_title"] = None
            try:
                job_post["job_description"] = job_soup.find( "div", {"class":"show-more-less-html__markup show-more-less-html__markup--clamp-after-5 relative overflow-hidden" }).text.strip()
            except: 
                job_post["job_description"] = None
            try:
                job_post["days_ago_posted"] = job_soup.find( "span", {"class": "posted-time-ago__text topcard__flavor--metadata"}).text.strip()
            except:
                job_post["days_ago_posted"] = None
            try:
                job_post["num_applicants"] = job_soup.find( "span", {"class": "num-applicants__caption topcard__flavor--metadata topcard__flavor--bullet"}).text.strip()
            except:
                job_post["num_applicants"] = None
            job_list.append(job_post)

    
    

In [3]:
jobs_df = pd.DataFrame(job_list)
jobs_df

,company_name,job_title,job_description,days_ago_posted,num_applicants
0,TikTok,Data Science Graduate (TikTok-Product-Data Sci...,ResponsibilitiesTikTok is the leading destinat...,1 week ago,None
1,Fractal,Data Scientist,Data ScientistFractal Analytics is a strategic...,3 days ago,None
2,Netflix,"Machine Learning Engineer Intern, Summer 2025",Netflix is one of the world's leading entertai...,1 month ago,None
3,Google,"Data Scientist, Search Discovery, Research, Se...",Minimum qualifications:Master's degree in Stat...,3 weeks ago,None
4,Fractal,Data Scientist,Data Scientist Fractal Analytics is a strategi...,4 weeks ago,None
...,...,...,...,...,...
230,Avanciers,Data Scientist,"Location: Bay Area , CA / Reno, NV. No remoteD...",3 days ago,None
231,Adobe,2025 Intern - Data Scientist,Our CompanyChanging the world through digital ...,1 week ago,None
232,Moveworks,Machine Learning Engineer - LLM,Who We Are Moveworks is the universal AI copil...,3 weeks ago,None
233,Intuit,Data Scientist 2,OverviewIntuit is hiring a Data Scientist 2 to...,1 day ago,None
